# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [20]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [21]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,longquan,24.6719,102.1613,25.86,77,100,0.62,CN,1691302092
1,1,saint-philippe,-21.3585,55.7679,22.77,77,75,7.25,RE,1691302093
2,2,margaret river,-33.9500,115.0667,15.23,62,74,2.19,AU,1691302093
3,3,bethel,41.3712,-73.4140,21.17,82,0,1.54,US,1691301978
4,4,waitangi,-43.9535,-176.5597,8.23,61,85,8.67,NZ,1691302094


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [22]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [23]:
# Narrow down cities that fit criteria and drop any results with null values
#       A max temperature lower than 27 degrees but higher than 21
#       Wind speed less than 4.5 m/s
#       Zero cloudiness
ideal_weather_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5) & \
                                    (city_data_df["Max Temp"] < 27.0) & (city_data_df["Max Temp"] > 21.0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
print(ideal_weather_df.count())
ideal_weather_df.head()

City_ID       24
City          24
Lat           24
Lng           24
Max Temp      24
Humidity      24
Cloudiness    24
Wind Speed    24
Country       24
Date          24
dtype: int64


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,bethel,41.3712,-73.4140,21.17,82,0,1.54,US,1691301978
14,14,woodland park,40.8898,-74.1949,24.22,82,0,2.06,US,1691302098
39,39,labytnangi,66.6572,66.4183,26.13,47,0,2.79,RU,1691302109
130,130,laghouat,33.8000,2.8651,23.90,20,0,0.63,DZ,1691302016
135,135,santa cruz,-17.8000,-63.1667,21.70,88,0,3.60,BO,1691301998


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
print(len(hotel_df))
hotel_df.head()

24


C:\Users\andre\AppData\Local\Temp\ipykernel_25392\2374050864.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,82,
14,woodland park,US,40.8898,-74.1949,82,
39,labytnangi,RU,66.6572,66.4183,47,
130,laghouat,DZ,33.8000,2.8651,20,
135,santa cruz,BO,-17.8000,-63.1667,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
woodland park - nearest hotel: Holiday Inn Totowa Wayne
labytnangi - nearest hotel: Семь лиственниц
laghouat - nearest hotel: فندق سيشال
santa cruz - nearest hotel: Aviador Hotel Boutique
kirando - nearest hotel: Lakeshore Lodge
gafsa - nearest hotel: Hôtel moussa
fort bragg - nearest hotel: Airborne Inn Lodging
remire-montjoly - nearest hotel: Complexe Belova
diamantino - nearest hotel: Hotel kayaby
lugoff - nearest hotel: Travelodge
port hueneme - nearest hotel: No hotel found
sterling heights - nearest hotel: Wyndham Garden Hotel
shilovo - nearest hotel: мечта
borodinskiy - nearest hotel: Белая лошадь
las lomitas - nearest hotel: Hotel Lomitas
kharp - nearest hotel: Собь
cherry valley - nearest hotel: Holiday Inn Express & Suites Beaumont - Oak Valley
williams lake - nearest hotel: Coast Fraser Inn
porto velho - nearest hotel: Hotel Slaviero Essential Porto Velho
lebanon - nearest hotel: Ramada
alvorada - nearest hote

,City,Country,Lat,Lng,Humidity,Hotel Name
3,bethel,US,41.3712,-73.4140,82,Hampton Inn Danbury
14,woodland park,US,40.8898,-74.1949,82,Holiday Inn Totowa Wayne
39,labytnangi,RU,66.6572,66.4183,47,Семь лиственниц
130,laghouat,DZ,33.8000,2.8651,20,فندق سيشال
135,santa cruz,BO,-17.8000,-63.1667,88,Aviador Hotel Boutique
157,kirando,TZ,-7.4167,30.6000,45,Lakeshore Lodge
164,gafsa,TN,34.4250,8.7842,60,Hôtel moussa
172,fort bragg,US,35.1390,-79.0060,88,Airborne Inn Lodging
208,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
232,diamantino,BR,-14.4086,-56.4461,47,Hotel kayaby


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.5,
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)